# UFoE Phase 5b-2: ColabFold Multimer v2
## GCN4 Leucine Zipper — 3 Groups × 4 Analyses

| Group | 서열 (33aa) | 설계 |
|-------|-------------|------|
| A (WT) | RMKQLEDKVEELLSKNYHLENEVARLKKLVGER | 자연 균형 |
| B (곤↑) | LMKQLEDIVEELLSLNYHLEINEVALKKLLGEL | a,d=L/I 100% |
| C (건↑) | REKKLEDKEEKLLSKEYKLENEEAKLKKLEGKR | e=E,g=K 100% |

**⚠️ 실행 전 반드시: 런타임 > 런타임 유형 변경 > T4 GPU**

---

## 1단계: 환경 설치 (~5분)

ColabFold + 분석 도구 설치. 설치 완료 후 런타임 재시작이 필요할 수 있습니다.

In [ ]:
%%time
import os, sys

if not os.path.exists('/content/SETUP_DONE'):
    # GPU 확인
    print('=== GPU 확인 ===')
    !nvidia-smi --query-gpu=name,memory.total --format=csv,noheader

    # ColabFold 설치 (공식 방법)
    print('\n=== ColabFold 설치 ===')
    !pip install -q --no-warn-conflicts \
        'colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold'
    !pip install -q 'jax[cuda12]'
    !pip install -q biopython matplotlib

    # AlphaFold 파라미터 다운로드
    print('\n=== AlphaFold 파라미터 다운로드 ===')
    !python -m colabfold.download

    !touch /content/SETUP_DONE
    print('\n✅ 설치 완료!')
    print('⚠️ 런타임 재시작이 필요하면 재시작 후 이 셀부터 다시 실행하세요.')
else:
    print('✅ 이미 설치됨. Skip.')

# 설치 검증
print('\n=== 검증 ===')
import shutil
cb = shutil.which('colabfold_batch')
print(f'colabfold_batch: {cb}')
try:
    import jax
    print(f'JAX devices: {jax.devices()}')
except Exception as e:
    print(f'JAX error: {e}')

## 2단계: 서열 정의 + FASTA 생성

In [ ]:
import os, json, glob
import numpy as np
from pathlib import Path
from collections import Counter, defaultdict

# === 곤감리건 7-3-6-4 ===
AA_TO_TYPE = {
    'A':'gon','L':'gon','V':'gon','I':'gon','F':'gon','M':'gon','W':'gon',
    'G':'gam','C':'gam','P':'gam',
    'S':'ri','T':'ri','N':'ri','Q':'ri','H':'ri','Y':'ri',
    'E':'geon','D':'geon','K':'geon','R':'geon',
}
MAX_ASA = {
    'ALA':129,'ARG':274,'ASN':195,'ASP':193,'CYS':167,
    'GLN':225,'GLU':223,'GLY':104,'HIS':224,'ILE':197,
    'LEU':201,'LYS':236,'MET':224,'PHE':240,'PRO':159,
    'SER':155,'THR':172,'TRP':285,'TYR':263,'VAL':174,
}
HEPTAD = 'defgabcdefgabcdefgabcdefgabcdefga'
A_POS = [i for i,h in enumerate(HEPTAD) if h=='a']
D_POS = [i for i,h in enumerate(HEPTAD) if h=='d']
E_POS = [i for i,h in enumerate(HEPTAD) if h=='e']
G_POS = [i for i,h in enumerate(HEPTAD) if h=='g']
CORE_POS = sorted(A_POS + D_POS)

# === 확정 서열 ===
GROUPS = {
    'A': {'name':'GCN4_WT',   'seq':'RMKQLEDKVEELLSKNYHLENEVARLKKLVGER', 'desc':'WT GCN4'},
    'B': {'name':'GCN4_GON',  'seq':'LMKQLEDIVEELLSLNYHLEINEVALKKLLGEL', 'desc':'곤강화 a,d=L/I'},
    'C': {'name':'GCN4_SALT', 'seq':'REKKLEDKEEKLLSKEYKLENEEAKLKKLEGKR', 'desc':'건강화 e=E,g=K'},
}

OUTDIR = Path('/content/phase5b2')
FASTA_DIR = OUTDIR / 'fasta'
RESULT_DIR = OUTDIR / 'results'
for d in [OUTDIR, FASTA_DIR, RESULT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# FASTA 생성 + 서열 검증
print('=' * 60)
print('  확정 서열 (3 Groups)')
print('=' * 60)
for gn in ['A','B','C']:
    g = GROUPS[gn]
    seq = g['seq']
    n = len(seq)
    c = Counter(AA_TO_TYPE[aa] for aa in seq)
    # FASTA
    fp = FASTA_DIR / f"{g['name']}.fasta"
    with open(fp,'w') as f:
        f.write(f">{g['name']}\n{seq}:{seq}\n")
    print(f"\n  Group {gn}: {g['desc']}")
    print(f"  {seq} ({n}aa)")
    print(f"  곤={c['gon']/n:.0%} 건={c['geon']/n:.0%}")
    print(f"  FASTA: {fp}")

print(f'\n✅ FASTA 3개 생성 완료')

## 3단계: ColabFold 실행 (그룹별)

**한 그룹씩 실행합니다.** 각 ~10-15분 소요.

In [ ]:
%%time
# === Group A: WT ===
print('=' * 60)
print('  Group A (GCN4_WT) 실행')
print('=' * 60)
!colabfold_batch \
    /content/phase5b2/fasta/GCN4_WT.fasta \
    /content/phase5b2/results \
    --num-models 1 \
    --num-recycle 3 \
    --model-type alphafold2_multimer_v3

print('\n=== PDB 확인 ===')
!ls /content/phase5b2/results/*.pdb 2>/dev/null || echo '❌ PDB 없음'

In [ ]:
%%time
# === Group B: 곤강화 ===
print('=' * 60)
print('  Group B (GCN4_GON) 실행')
print('=' * 60)
!colabfold_batch \
    /content/phase5b2/fasta/GCN4_GON.fasta \
    /content/phase5b2/results \
    --num-models 1 \
    --num-recycle 3 \
    --model-type alphafold2_multimer_v3

print('\n=== PDB 확인 ===')
!ls /content/phase5b2/results/*.pdb 2>/dev/null || echo '❌ PDB 없음'

In [ ]:
%%time
# === Group C: 건강화 ===
print('=' * 60)
print('  Group C (GCN4_SALT) 실행')
print('=' * 60)
!colabfold_batch \
    /content/phase5b2/fasta/GCN4_SALT.fasta \
    /content/phase5b2/results \
    --num-models 1 \
    --num-recycle 3 \
    --model-type alphafold2_multimer_v3

print('\n=== 전체 PDB 확인 ===')
!ls -la /content/phase5b2/results/*.pdb 2>/dev/null || echo '❌ PDB 없음'
print('\n=== 전체 결과 파일 ===')
!ls /content/phase5b2/results/

### ⚠️ ColabFold 실패 시 대안

위 셀이 실패하면 아래 셀로 **Python API 직접 호출**을 시도하세요.

In [ ]:
# === 대안: Python API로 직접 실행 ===
# colabfold_batch CLI가 실패할 경우 이 셀을 실행하세요

try:
    from colabfold.batch import run, set_model_type
    from colabfold.download import default_data_dir
    from colabfold.utils import setup_logging
    import logging

    setup_logging(Path('/content/phase5b2/log.txt'))

    for gn in ['A', 'B', 'C']:
        g = GROUPS[gn]
        fasta = str(FASTA_DIR / f"{g['name']}.fasta")
        result_dir = str(RESULT_DIR)

        print(f'\n{"="*60}')
        print(f'  Group {gn} ({g["name"]}) — Python API')
        print(f'{"="*60}')

        run(
            queries=fasta,
            result_dir=result_dir,
            num_models=1,
            num_recycles=3,
            model_type='alphafold2_multimer_v3',
            data_dir=default_data_dir,
        )
        print(f'  ✅ {g["name"]} 완료')

    print('\n=== 전체 PDB 확인 ===')
    !ls -la /content/phase5b2/results/*.pdb 2>/dev/null || echo '❌ PDB 없음'

except ImportError as e:
    print(f'❌ ColabFold API import 실패: {e}')
    print('\n1단계 설치를 먼저 실행하세요.')
except Exception as e:
    print(f'❌ 실행 오류: {e}')
    import traceback
    traceback.print_exc()

### ⚠️ 최후 대안: sokrypton 공식 노트북 사용

위 방법이 모두 실패하면:
1. [공식 ColabFold 노트북](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb) 열기
2. `query_sequence`에 아래 서열 입력 (3번 반복):
```
Group A: RMKQLEDKVEELLSKNYHLENEVARLKKLVGER:RMKQLEDKVEELLSKNYHLENEVARLKKLVGER
Group B: LMKQLEDIVEELLSLNYHLEINEVALKKLLGEL:LMKQLEDIVEELLSLNYHLEINEVALKKLLGEL  
Group C: REKKLEDKEEKLLSKEYKLENEEAKLKKLEGKR:REKKLEDKEEKLLSKEYKLENEEAKLKKLEGKR
```
3. `model_type` = `alphafold2_multimer_v3`
4. 실행 후 PDB 다운로드 → 이 노트북의 5단계에서 업로드

## 4단계: PDB 파일 로드

In [ ]:
# PDB 자동 탐색 + 수동 경로 지원

pdb_files = {}

for gn in ['A','B','C']:
    name = GROUPS[gn]['name']
    found = None

    # 탐색 패턴 (넓은 범위)
    for pattern in [
        f'/content/phase5b2/results/{name}*rank*001*.pdb',
        f'/content/phase5b2/results/{name}*.pdb',
        f'/content/phase5b2/results/*{name}*.pdb',
        f'/content/**/results/{name}*.pdb',
        f'/content/**/{name}*.pdb',
    ]:
        matches = sorted(glob.glob(pattern, recursive=True))
        if matches:
            found = matches[0]
            break

    if found:
        pdb_files[gn] = found
        print(f'  Group {gn}: ✅ {found}')
    else:
        print(f'  Group {gn}: ❌ NOT FOUND')

print(f'\n  PDB: {len(pdb_files)}/3 발견')

if len(pdb_files) < 3:
    print('\n  결과 디렉토리 전체 내용:')
    !find /content/phase5b2 -type f 2>/dev/null
    print('\n  ⚠️ PDB가 없으면 3단계를 다시 확인하세요.')
    print('  또는 아래 셀에서 수동으로 경로를 지정하세요.')

In [ ]:
# === 수동 PDB 경로 지정 (필요시) ===
# 공식 ColabFold 노트북에서 PDB를 다운로드한 경우,
# Colab 왼쪽 패널에서 파일 업로드 후 경로 지정:

# pdb_files['A'] = '/content/GCN4_WT_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb'
# pdb_files['B'] = '/content/GCN4_GON_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb'
# pdb_files['C'] = '/content/GCN4_SALT_unrelaxed_rank_001_alphafold2_multimer_v3_model_1_seed_000.pdb'

print('현재 PDB:')
for gn in ['A','B','C']:
    p = pdb_files.get(gn, 'NOT SET')
    ok = os.path.exists(p) if p != 'NOT SET' else False
    print(f'  {gn}: {p} {"✅" if ok else "❌"}')

## 5단계: 4가지 분석 실행

PDB가 3개 모두 있어야 실행됩니다.

In [ ]:
from Bio.PDB import PDBParser, ShrakeRupley

parser = PDBParser(QUIET=True)

# ═══════════════════════════════════════
# 분석 함수 정의
# ═══════════════════════════════════════

def get_plddt(pdb_path):
    s = parser.get_structure('s', pdb_path)
    chains = list(s[0].get_chains())
    result = {'n_chains': len(chains)}
    all_p = []
    for ch in chains:
        ps = [a.bfactor for r in ch for a in r if a.name=='CA']
        if ps and max(ps) <= 1.0:
            ps = [p*100 for p in ps]
        result[ch.id] = round(np.mean(ps),1) if ps else 0
        all_p.extend(ps)
    result['total'] = round(np.mean(all_p),1) if all_p else 0
    return result


def detect_salt_bridges(pdb_path, seq, cutoff=4.0):
    s = parser.get_structure('s', pdb_path)
    chains = list(s[0].get_chains())
    if len(chains) < 2:
        return {'total':0, 'canonical_count':0, 'all':[], 'canonical':[]}
    neg = {'GLU':['OE1','OE2'], 'ASP':['OD1','OD2']}
    pos = {'LYS':['NZ'], 'ARG':['NH1','NH2']}
    ra, rb = list(chains[0].get_residues()), list(chains[1].get_residues())
    bridges = []
    for i, r1 in enumerate(ra):
        for j, r2 in enumerate(rb):
            pairs = []
            if r1.get_resname() in neg and r2.get_resname() in pos:
                pairs = [(neg[r1.get_resname()], pos[r2.get_resname()])]
            elif r1.get_resname() in pos and r2.get_resname() in neg:
                pairs = [(pos[r1.get_resname()], neg[r2.get_resname()])]
            for a_atoms, b_atoms in pairs:
                md = 999
                for a in a_atoms:
                    for b in b_atoms:
                        if a in r1 and b in r2:
                            d = r1[a] - r2[b]
                            if d < md: md = d
                if md < cutoff:
                    ha = HEPTAD[i] if i<len(HEPTAD) else '?'
                    hb = HEPTAD[j] if j<len(HEPTAD) else '?'
                    canon = ha in('e','g') and hb in('e','g')
                    bridges.append({
                        'i':i+1,'j':j+1,
                        'aa_i':seq[i] if i<len(seq) else '?',
                        'aa_j':seq[j] if j<len(seq) else '?',
                        'hi':ha,'hj':hb,'dist':round(md,2),'canonical':canon})
    # dedup
    seen = set()
    unique = []
    for b in sorted(bridges, key=lambda x:x['dist']):
        k = tuple(sorted([b['i'],b['j']]))
        if k not in seen:
            seen.add(k); unique.append(b)
    canon = [b for b in unique if b['canonical']]
    return {'total':len(unique),'canonical_count':len(canon),'all':unique,'canonical':canon}


def core_packing(pdb_path, seq):
    s = parser.get_structure('s', pdb_path)
    chains = list(s[0].get_chains())
    if len(chains)<2: return {'error':'<2 chains'}
    ra, rb = list(chains[0].get_residues()), list(chains[1].get_residues())
    def cb(r):
        if 'CB' in r: return r['CB'].get_vector().get_array()
        if 'CA' in r: return r['CA'].get_vector().get_array()
        return None
    dists = {'a_a':[],'d_d':[],'a_d':[]}
    same = []
    na, nb = min(len(seq),len(ra)), min(len(seq),len(rb))
    for p in A_POS:
        if p>=na or p>=nb: continue
        a,b = cb(ra[p]), cb(rb[p])
        if a is not None and b is not None:
            d = np.linalg.norm(a-b)
            dists['a_a'].append(d)
            same.append({'t':'a','pos':p+1,'aa':seq[p],'dist':round(d,2)})
    for p in D_POS:
        if p>=na or p>=nb: continue
        a,b = cb(ra[p]), cb(rb[p])
        if a is not None and b is not None:
            d = np.linalg.norm(a-b)
            dists['d_d'].append(d)
            same.append({'t':'d','pos':p+1,'aa':seq[p],'dist':round(d,2)})
    for pa in A_POS:
        for pd in D_POS:
            if pa>=na or pd>=nb: continue
            a,b = cb(ra[pa]), cb(rb[pd])
            if a is not None and b is not None:
                dists['a_d'].append(np.linalg.norm(a-b))
    stats = {}
    for k,v in dists.items():
        if v: stats[k] = {'min':round(min(v),2),'mean':round(np.mean(v),2),'std':round(np.std(v),2)}
    return {'stats':stats, 'same_heptad':same}


def crossing_angle(pdb_path):
    s = parser.get_structure('s', pdb_path)
    chains = list(s[0].get_chains())
    if len(chains)<2: return {'error':'<2 chains'}
    def axis(ch):
        ca = np.array([r['CA'].get_vector().get_array() for r in ch if 'CA' in r])
        if len(ca)<5: return None, None
        c = ca.mean(0); cov = np.cov((ca-c).T)
        ev, evec = np.linalg.eigh(cov)
        return evec[:,ev.argmax()], c
    ax_a, ca = axis(chains[0])
    ax_b, cb_ = axis(chains[1])
    if ax_a is None or ax_b is None: return {'error':'not enough CA'}
    cos = np.clip(np.dot(ax_a,ax_b)/(np.linalg.norm(ax_a)*np.linalg.norm(ax_b)),-1,1)
    ang = np.degrees(np.arccos(abs(cos)))
    return {'angle':round(ang,1),'parallel':bool(cos>0),'dist':round(np.linalg.norm(ca-cb_),1)}


def burial_ratio(pdb_path, seq):
    s = parser.get_structure('s', pdb_path)
    sr = ShrakeRupley(); sr.compute(s[0], level='R')
    ch = list(s[0].get_chains())
    if not ch: return {'error':'no chains'}
    res = list(ch[0].get_residues())
    n = min(len(seq), len(res))
    rsa = [res[i].sasa / MAX_ASA.get(res[i].get_resname(),200) for i in range(n)]
    core = [rsa[i] for i in CORE_POS if i<n]
    surf = [rsa[i] for i in range(n) if i not in CORE_POS]
    if not core or not surf: return {'error':'too few'}
    cm, sm = np.mean(core), np.mean(surf)
    return {'core':round(cm,4),'surface':round(sm,4),'ratio':round(cm/sm if sm>0 else 999,4)}

print('✅ 분석 함수 4개 로드 완료')

In [ ]:
# ═══════════════════════════════════════
# 전체 분석 실행
# ═══════════════════════════════════════

results = {}

for gn in ['A','B','C']:
    if gn not in pdb_files:
        print(f'  Group {gn}: ❌ SKIP (no PDB)')
        continue
    g = GROUPS[gn]
    pdb = pdb_files[gn]
    seq = g['seq']
    print(f'\n{"="*60}')
    print(f'  Group {gn}: {g["name"]}')
    print(f'{"="*60}')

    r = {}

    # pLDDT
    r['plddt'] = get_plddt(pdb)
    print(f'  pLDDT: {r["plddt"]}')

    # Salt bridge
    r['salt'] = detect_salt_bridges(pdb, seq)
    print(f'  Salt bridges: {r["salt"]["total"]} total, {r["salt"]["canonical_count"]} canonical')
    for b in r['salt']['canonical'][:5]:
        print(f'    {b["aa_i"]}{b["i"]}({b["hi"]})—{b["aa_j"]}{b["j"]}({b["hj"]}) {b["dist"]}A ★')

    # Core packing
    r['core'] = core_packing(pdb, seq)
    if 'stats' in r['core']:
        for k in ['d_d','a_a','a_d']:
            s = r['core']['stats'].get(k)
            if s: print(f'  {k}: min={s["min"]}A mean={s["mean"]}A')
        for sh in sorted(r['core']['same_heptad'], key=lambda x:x['dist'])[:5]:
            print(f'    {sh["t"]}{sh["pos"]}: {sh["aa"]}↔{sh["aa"]} = {sh["dist"]}A')

    # Crossing angle
    r['angle'] = crossing_angle(pdb)
    if 'angle' in r['angle']:
        par = '∥' if r['angle']['parallel'] else '⊗'
        ok = '✅' if 18<=r['angle']['angle']<=25 else '⚠️'
        print(f'  Crossing: {r["angle"]["angle"]}° {par} dist={r["angle"]["dist"]}A {ok}')

    # Burial
    r['burial'] = burial_ratio(pdb, seq)
    if 'ratio' in r['burial']:
        ok = '✅' if r['burial']['ratio']<0.6 else '❌'
        print(f'  Burial: {r["burial"]["ratio"]} {ok}')

    results[gn] = r

print(f'\n✅ 분석 완료: {len(results)}/3 그룹')

## 6단계: 종합 비교 + 판정

In [ ]:
print('=' * 80)
print('  UFoE Phase 5b-2: ColabFold Multimer 종합')
print('=' * 80)

# ESMFold 기준선
ESM = {
    'A':{'plddt':69.0,'salt':0,'salt_eg':0,'dd':7.24,'angle':24.8,'burial':0.529},
    'B':{'plddt':80.9,'salt':0,'salt_eg':0,'dd':4.82,'angle':20.2,'burial':0.535},
    'C':{'plddt':71.7,'salt':5,'salt_eg':1,'dd':15.53,'angle':11.7,'burial':1.345},
}

# 비교표
rows = [
    ('pLDDT',         lambda g: results.get(g,{}).get('plddt',{}).get('total','—')),
    ('Salt (total)',   lambda g: results.get(g,{}).get('salt',{}).get('total','—')),
    ('Salt (e-g\')',  lambda g: results.get(g,{}).get('salt',{}).get('canonical_count','—')),
    ('d↔d\' min(A)',  lambda g: results.get(g,{}).get('core',{}).get('stats',{}).get('d_d',{}).get('min','—')),
    ('Angle (°)',      lambda g: results.get(g,{}).get('angle',{}).get('angle','—')),
    ('Burial ratio',   lambda g: results.get(g,{}).get('burial',{}).get('ratio','—')),
]

print(f'\n  {"":20s} | {"A (WT)":>12s} | {"B (곤↑)":>12s} | {"C (건↑)":>12s}')
print(f'  {"-"*65}')
for label, fn in rows:
    vals = [fn(g) for g in ['A','B','C']]
    print(f'  {label:20s} | {str(vals[0]):>12s} | {str(vals[1]):>12s} | {str(vals[2]):>12s}')

# ESMFold 대비
print(f'\n  {"":20s} | {"ESMFold":>12s} | {"ColabFold":>12s}')
print(f'  {"-"*50}')
for gn in ['A','B','C']:
    print(f'\n  == Group {gn} ==')
    e = ESM[gn]
    r = results.get(gn, {})
    cf_plddt = r.get('plddt',{}).get('total','—')
    cf_salt = r.get('salt',{}).get('total','—')
    cf_dd = r.get('core',{}).get('stats',{}).get('d_d',{}).get('min','—')
    cf_ang = r.get('angle',{}).get('angle','—')
    print(f'  {"pLDDT":20s} | {e["plddt"]:>12.1f} | {str(cf_plddt):>12s}')
    print(f'  {"Salt total":20s} | {e["salt"]:>12d} | {str(cf_salt):>12s}')
    print(f'  {"d↔d min":20s} | {e["dd"]:>12.2f} | {str(cf_dd):>12s}')
    print(f'  {"Angle":20s} | {e["angle"]:>12.1f} | {str(cf_ang):>12s}')

# ═══ 판정 ═══
print(f'\n{"="*80}')
print('  3원리 판정')
print(f'{"="*80}')

j = {}

# 1. 곤 코어
dd_a = results.get('A',{}).get('core',{}).get('stats',{}).get('d_d',{}).get('min',999)
dd_b = results.get('B',{}).get('core',{}).get('stats',{}).get('d_d',{}).get('min',999)
if dd_a != 999 and dd_b != 999:
    j['gon'] = dd_b <= dd_a
    print(f'  [1] 곤 코어: B d↔d\' {dd_b}A {"<=" if j["gon"] else ">"} A {dd_a}A → {"✅" if j["gon"] else "❌"}')
else:
    print('  [1] 곤 코어: 데이터 부족')

# 2. 건 표면
sa = results.get('A',{}).get('salt',{}).get('total',0)
sc = results.get('C',{}).get('salt',{}).get('total',0)
ba = results.get('A',{}).get('burial',{}).get('ratio',0)
bc = results.get('C',{}).get('burial',{}).get('ratio',0)
j['geon'] = sc >= sa and bc > ba
print(f'  [2] 건 표면: Salt A={sa} C={sc}, Burial A={ba} C={bc} → {"✅" if j["geon"] else "❌"}')

# 3. 균형
ang_a = results.get('A',{}).get('angle',{}).get('angle',999)
if ang_a != 999:
    j['balance'] = 15 <= ang_a <= 30
    print(f'  [3] 균형: WT angle={ang_a}° → {"✅" if j["balance"] else "❌"}')
else:
    print('  [3] 균형: 데이터 부족')

pc = sum(1 for v in j.values() if v)
print(f'\n  결과: {pc}/{len(j)} PASS')
if pc == len(j) == 3:
    print('  🎉 3원리 모두 검증 성공!')

## 7단계: 결과 저장 + 다운로드

In [ ]:
# JSON 저장
output = {
    'experiment': 'UFoE Phase 5b-2 ColabFold Multimer v2',
    'groups': {gn: {'seq': GROUPS[gn]['seq'], **results.get(gn,{})} for gn in ['A','B','C']},
    'judgments': j,
    'esmfold_ref': ESM,
}
jp = OUTDIR / 'phase5b2_results.json'
with open(jp,'w') as f:
    json.dump(output, f, ensure_ascii=False, indent=2, default=str)
print(f'JSON: {jp}')

# Colab 다운로드
try:
    from google.colab import files
    files.download(str(jp))
    for gn in ['A','B','C']:
        if gn in pdb_files and os.path.exists(pdb_files[gn]):
            files.download(pdb_files[gn])
    print('📥 다운로드 시작')
except:
    print('수동 다운로드 필요')